In [ ]:
#Train test split to find model performance
#K-fold cross validation
#GrindSearchCV for hyperparameter tuning
#RandomizedSearchCV

#our point is choosing best model


In [1]:
from sklearn import svm,datasets
iris = datasets.load_iris()

In [5]:
dir(iris)

['DESCR',
 'data',
 'data_module',
 'feature_names',
 'filename',
 'frame',
 'target',
 'target_names']

In [14]:
import pandas as pd
df = pd.DataFrame(iris.data, columns = iris.feature_names)
df['flower'] = iris.target
df['flower'] = df['flower'].apply(lambda x: iris.target_names[x])
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),flower
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [15]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(iris.data,iris.target, test_size = 0.3)

In [19]:
model = svm.SVC(kernel ='rbf', C = 30, gamma = 'auto')
model.fit(X_train,y_train)
model.score(X_test,y_test)

0.9111111111111111

In [20]:
#Define function about multiClassification
def function(model,X_train,X_test,y_train,y_test):
    model.fit(X_train,y_train)
    return model.score(X_test,y_test)

function(svm.SVC(), X_train,X_test,y_train,y_test)

0.9111111111111111

In [29]:
#Cross Validation
from sklearn.model_selection import cross_val_score
cross_val_score(svm.SVC(kernel ='linear', C=10, gamma='auto' ),iris.data, iris.target, cv=5)

array([1.        , 1.        , 0.9       , 0.96666667, 1.        ])

In [31]:
cross_val_score(svm.SVC(kernel = 'rbf', C= 10), iris.data,iris.target)

array([0.96666667, 1.        , 0.96666667, 0.96666667, 1.        ])

In [32]:
#Same Cross Validation but with for loop

import numpy as np
kernels = ['linear', 'rbf']
C = [10,20,30]
avg_scores = {}

for kval in kernels:
    for cval in C:
        cv_scores = cross_val_score(svm.SVC(kernel = kval, C= cval),iris.data,iris.target)
        avg_scores[kval + '_' + str(cval)] = np.average(cv_scores)
        
avg_scores

#this apporach not convinient, we have API called GridSearchCV

{'linear_10': 0.9733333333333334,
 'linear_20': 0.9666666666666666,
 'linear_30': 0.96,
 'rbf_10': 0.9800000000000001,
 'rbf_20': 0.9800000000000001,
 'rbf_30': 0.9733333333333334}

In [35]:
#GridSearchCV
from sklearn.model_selection import GridSearchCV
clf = GridSearchCV(svm.SVC(gamma = 'auto'),{'C':[10,20,30], 'kernel': ['rbf','linear']}, return_train_score=False)

clf.fit(iris.data,iris.target)
clf.cv_results_

#hard to read, lets make simple with pandas

{'mean_fit_time': array([0.00060291, 0.00060234, 0.00060525, 0.00080094, 0.00040059,
        0.00040002]),
 'std_fit_time': array([0.00049229, 0.00049187, 0.00049431, 0.00040051, 0.00049062,
        0.00048992]),
 'mean_score_time': array([0.0004003 , 0.0004003 , 0.00040016, 0.00020013, 0.0006022 ,
        0.00039845]),
 'std_score_time': array([0.00049027, 0.00049027, 0.00049013, 0.00040026, 0.00049171,
        0.000488  ]),
 'param_C': masked_array(data=[10, 10, 20, 20, 30, 30],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_kernel': masked_array(data=['rbf', 'linear', 'rbf', 'linear', 'rbf', 'linear'],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'C': 10, 'kernel': 'rbf'},
  {'C': 10, 'kernel': 'linear'},
  {'C': 20, 'kernel': 'rbf'},
  {'C': 20, 'kernel': 'linear'},
  {'C': 30, 'kernel': 'rbf'},
  {'C': 30, 'kernel': 'linear'

In [37]:
df = pd.DataFrame(clf.cv_results_)
df.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.000603,0.000492,0.000400,0.000490,10,rbf,"{'C': 10, 'kernel': 'rbf'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
1,0.000602,0.000492,0.000400,0.000490,10,linear,"{'C': 10, 'kernel': 'linear'}",1.000000,1.0,0.900000,0.966667,1.0,0.973333,0.038873,2
2,0.000605,0.000494,0.000400,0.000490,20,rbf,"{'C': 20, 'kernel': 'rbf'}",0.966667,1.0,0.900000,0.966667,1.0,0.966667,0.036515,3
3,0.000801,0.000401,0.000200,0.000400,20,linear,"{'C': 20, 'kernel': 'linear'}",1.000000,1.0,0.900000,0.933333,1.0,0.966667,0.042164,4
4,0.000401,0.000491,0.000602,0.000492,30,rbf,"{'C': 30, 'kernel': 'rbf'}",0.966667,1.0,0.900000,0.933333,1.0,0.960000,0.038873,5


In [38]:
df[['param_C', 'param_kernel', 'mean_test_score']]

,param_C,param_kernel,mean_test_score
0,10,rbf,0.980000
1,10,linear,0.973333
2,20,rbf,0.966667
3,20,linear,0.966667
4,30,rbf,0.960000
5,30,linear,0.960000


In [39]:
dir(clf)

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_check_feature_names',
 '_check_n_features',
 '_check_refit_for_multimetric',
 '_estimator_type',
 '_format_results',
 '_get_param_names',
 '_get_tags',
 '_more_tags',
 '_pairwise',
 '_repr_html_',
 '_repr_html_inner',
 '_repr_mimebundle_',
 '_required_parameters',
 '_run_search',
 '_select_best_index',
 '_validate_data',
 'best_estimator_',
 'best_index_',
 'best_params_',
 'best_score_',
 'classes_',
 'cv',
 'cv_results_',
 'decision_function',
 'error_score',
 'estimator',
 'fit',
 'get_params',
 'inverse_transform',
 'multimetric_',
 'n_features_

In [43]:
#RandomizedSearchCV
from sklearn.model_selection import RandomizedSearchCV
rs = RandomizedSearchCV(svm.SVC(),{'C':[1,20,30], 'kernel': ['rbf','linear']},return_train_score=False, n_iter=2)

rs.fit(iris.data,iris.target)
pd.DataFrame(rs.cv_results_)[['param_C','param_kernel','mean_test_score']]

,param_C,param_kernel,mean_test_score
0,30,rbf,0.973333
1,30,linear,0.960000


In [49]:
##
##How choose a best model
##
##

from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

model_params = {
    'svm': {
        'model': svm.SVC(gamma='auto'),
        'params' : {
            'C': [1,10,20],
            'kernel': ['rbf','linear']
        }  
    },
    'random_forest': {
        'model': RandomForestClassifier(),
        'params' : {
            'n_estimators': [1,5,10]
        }
    },
    'logistic_regression' : {
        'model': LogisticRegression(solver='liblinear',multi_class='auto'),
        'params': {
            'C': [1,5,10]
        }
    }
}

scores = []
for model_name, mp in model_params.items():
    clf=GridSearchCV(mp['model'], mp['params'], return_train_score=False)
    clf.fit(iris.data,iris.target)
    scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })

In [50]:
df = pd.DataFrame(scores,columns=['model','best_score', 'best_params'])
df

,model,best_score,best_params
0,svm,0.980000,"{'C': 1, 'kernel': 'rbf'}"
1,random_forest,0.960000,{'n_estimators': 5}
2,logistic_regression,0.966667,{'C': 5}


In [55]:
####
####Exercise: Machine Learning Finding Optimal Model and Hyperparameters
####For digits dataset in sklearn.dataset, please try following classifiers and 
####find out the one that gives best performance.
####Also find the optimal parameters for that classifier.


from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier

from sklearn import datasets
digits = datasets.load_digits()

In [56]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(digits.data,digits.target,test_size =0.3)

In [57]:
#Use GridSearchCV
model_params = {
    'svm': {
        'model': svm.SVC(gamma='auto'),
        'params' : {
            'C': [1,10,20],
            'kernel': ['rbf','linear']
        }  
    },
    'random_forest': {
        'model': RandomForestClassifier(),
        'params' : {
            'n_estimators': [1,5,10]
        }
    },
    'logistic_regression' : {
        'model': LogisticRegression(solver='liblinear',multi_class='auto'),
        'params': {
            'C': [1,5,10]
        }
    },
     'GaussianNB' : {
        'model': GaussianNB(),
       'params': {}
    },
    'MultinomialNB' : {
        'model': MultinomialNB(),
        'params': {}
    },
     'DecisionTree' : {
        'model': DecisionTreeClassifier(),
        'params': {
           'criterion': ['gini','entropy'],
        }
    }
}

In [62]:
#Grid SearchCV
from sklearn.model_selection import GridSearchCV

for model_name, mp in model_params.items():
    clf= GridSearchCV(mp['model'],mp['params'],cv=5,return_train_score=False)
    clf.fit(digits.data,digits.target)
    scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })

In [63]:
df = pd.DataFrame(scores,columns=['model','best_score', 'best_params'])
df

,model,best_score,best_params
0,svm,0.980000,"{'C': 1, 'kernel': 'rbf'}"
1,random_forest,0.960000,{'n_estimators': 5}
2,logistic_regression,0.966667,{'C': 5}
3,svm,0.947697,"{'C': 1, 'kernel': 'linear'}"
4,random_forest,0.900972,{'n_estimators': 10}
5,logistic_regression,0.922114,{'C': 1}
6,GaussianNB,0.806928,{}
7,MultinomialNB,0.870350,{}
8,DecisionTree,0.815823,{'criterion': 'entropy'}
9,svm,0.947697,"{'C': 1, 'kernel': 'linear'}"
